In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = np.load('final-data/3600.npz')

In [3]:
earth = data['rho'] == 0
not_earth = ~earth

features = ['Bx', 'By', 'Bz', 'Ex', 'Ey', 'Ez', 'rho', 'vx', 'vy', 'vz', 'agyrotropy', 'anisotropy']

In [4]:
def standardize(feature):
    avg = np.mean(feature)
    std = np.std(feature)
    return (feature - avg) / std

In [5]:
data_dict = {}
for feature in features:
    current_feature = data[feature].copy()
    current_feature[not_earth] = standardize(current_feature[not_earth])
    current_feature[earth] = 0
    data_dict[f'{feature}_standardized'] = current_feature

In [6]:
y = data['labeled_domain']

In [7]:
y_i = np.argwhere(y)[:,0]
x_i = np.argwhere(y)[:,1]

In [8]:
def normalize(feature, abs_val=None):
    if abs_val is not None:
        max_val = np.max(abs_val)
    else:
        max_val = np.max(feature)
    return feature / max_val

In [9]:
def euclidian(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

In [ ]:
for feature in features:
    abs_E = euclidian(data['Ex'], data['Ey'], data['Ez'])
    abs_B = euclidian(data['Bx'], data['By'], data['Bz'])
    abs_v = euclidian(data['vx'], data['vy'], data['vz'])

    for feature in features:
        current_feature = data[feature].copy()
        if feature.startswith('E'):
            current_feature[not_earth] = normalize(current_feature[not_earth], abs_E[not_earth])
        elif feature.startswith('B'):
            current_feature[not_earth] = normalize(current_feature[not_earth], abs_B[not_earth])
        elif feature.startswith('v'):
            current_feature[not_earth] = normalize(current_feature[not_earth], abs_v[not_earth])
        else:
            current_feature[not_earth] = normalize(current_feature[not_earth])
        current_feature[earth] = 0
        data_dict[f'{feature}_normalized'] = current_feature

In [ ]:
data_dict.keys()

In [ ]:
for feature in features:
    fig = plt.figure(figsize=(22, 6))

    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)

    
    c1 = ax1.imshow(data[feature])
    ax1.scatter(x_i, y_i, marker='x', color='red')
    ax1.set_title(feature)
    fig.colorbar(c1, ax=ax1)
    
    c2 = ax2.imshow(data_dict[f'{feature}_standardized'])
    ax2.scatter(x_i, y_i, marker='x', color='red')
    ax2.set_title(f'{feature} standardized')
    fig.colorbar(c2, ax=ax2)
    
    c3 = ax3.imshow(data_dict[f'{feature}_normalized'])
    ax3.scatter(x_i, y_i, marker='x', color='red')
    ax3.set_title(f'{feature} normalized')
    fig.colorbar(c3, ax=ax3)
    
    plt.show()